# 1 Extracting genes and variants

In [ ]:
import pandas as pd
import math
import numpy as np

In [ ]:
df = pd.read_csv("data\model_output\extracted_snippets.csv")
df = df.to_numpy() # WBPaper ID, Method, * Gene-Variant combo , Mutation, Sentence

In [ ]:
text = pd.read_csv("data\model_output\extracted_snippets.csv")
text = text.to_numpy() # WBPaper ID, Method, * Gene-Variant combo , Mutation, Sentence

wb_genes = np.load('data\gsoc\wbtools\all_gene_names.npy')
all_genes = Path('data\gsoc\wbtools\genes.txt').read_text().split('\n')
for g in wb_genes: all_genes.append(g)
all_genes = [g for g in all_genes if len(g) > 1]
all_genes = list(set(all_genes))

OPENING_CLOSING_REGEXES = [r'((?:^|[\s\(\[\'"/,;\-])', r'(?:^|[\s\(\[\'"/,;\-]))']

all_genes = OPENING_CLOSING_REGEXES[0] + '|'.join(all_genes) + OPENING_CLOSING_REGEXES[1]
all_genes = [re.compile(r,re.IGNORECASE) for r in [all_genes]]

allele_designations = np.load('data\gsoc\wbtools\wb_allele_designations.npy')
alleles_variations = np.load('data\gsoc\wbtools\wb_alleles_variations.npy') 
DB_VAR_REGEX = r'(?:^|[\s\(\[\'"/,;\-])({designations}|m|p|ts|gf|lf|d|sd|am|cs)([0-9]+)(?:^|[\s\(\[\'"/,;\-])'
var_regex_1 = DB_VAR_REGEX.format(designations="|".join(allele_designations))
var_regex_2 = OPENING_CLOSING_REGEXES[0] + '|'.join(alleles_variations) + OPENING_CLOSING_REGEXES[1]

all_var = [re.compile(r,re.IGNORECASE) for r in [var_regex_1, var_regex_2]]

final = []
total = len(text)
for i, mut, sent in enumerate(text[:, -2]):
    if math.isnan(mut):
        continue
    sent = ': '..join()
    if (i+1) % 50 == 0: print(f"{i+1}>{total-1}", end = " ")
    genes = []
    for regex in all_genes:      
        for m in regex.finditer(sent):
            span = (m.start(0), m.end(0))    
            raw = (sent[span[0]:span[1]])
            raw = raw[1:] if not raw[0].isalnum() else raw
            raw = raw[:-1] if not raw[-1].isalnum() else raw
            genes.append(raw.strip())
    vars = []
    for regex in all_var:      
        for m in regex.finditer(sent):
            span = (m.start(0), m.end(0))    
            raw = (sent[span[0]:span[1]])
            raw = raw[1:] if not raw[0].isalnum() else raw
            raw = raw[:-1] if not raw[-1].isalnum() else raw
            vars.append(raw.strip())
    if genes:
        genes  = list(set(genes))
        genes = ", ".join(genes)
    else:
        genes = ''
    if vars:
        vars  = list(set(vars))
        vars = ", ".join(vars)
    else:
        vars = ''
    final.append([text[i,0], text[i,1], genes, vars, text[i,2], text[i,3], text[i,4]])

In [ ]:
np.save(r'data/model_output/results.npy', final)

data = pd.DataFrame(final[:], columns=['WBPaper ID', 'Method', 'Gene name', 'Variant name', '*Gene-Variant combo ', 'Mutations', 'Sentence'])
data.to_csv("./data/model_output/extracted_snippets.csv", index=False, encoding='utf-8')

# 2 Normalizing mutations